In [1]:
from taxcalc import *
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, push_notebook
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from bokeh.models.widgets import Select
from bokeh.models.annotations import LabelSet
from bokeh.layouts import row
from notebookfunctions import distribution, percentile, weighted_sum
from ipywidgets import interactive, Dropdown
from IPython.display import display
import pandas as pd
import numpy as np
import copy
output_notebook()

Loading BokehJS ...

In [2]:
ben_name_list = ['Medicare', 'Medicaid', 'Veterans Benefits', 'SNAP', 'SSI', 'Social Security']
ben_part_names = ['mcare_part', 'mcaid_part', 'vb_part', 'snap_part', 'ssi_part', 'ss_part']
years = [year for year in range(2014, 2027)]
participation_index = [i for i in range(1, 101)]
bin_axis = ['{}-{}%'.format(i - 10, i) for i in range(10, 110, 10)]

##### Functions used for plot updating

In [3]:
# function to update the total benefit plot
def total_bar_update(ben):
    init_bar.data_source.data['top'] = ben_total_cds.data[ben]
    total_fig.title.text = 'Total {} Benefit by Year'.format(ben)
    push_notebook(handle=value_handle)

In [4]:
# function to update participation rate year
def participation_rate_update(year):
    mcare_part.data_source.data['Medicare'] = participation_rates[year]['Medicare']
    mcaid_part.data_source.data['Medicaid'] = participation_rates[year]['Medicaid']
    ss_part.data_source.data['Social Security'] = participation_rates[year]['Social Security']
    snap_part.data_source.data['SNAP'] = participation_rates[year]['SNAP']
    ssi_part.data_source.data['SSI'] = participation_rates[year]['SSI']
    vb_part.data_source.data['Veterans Benefits'] = participation_rates[year]['Veterans Benefits']
    part_fig.title.text = 'Participation Rates by AGI Percentile for {}'.format(year)
    push_notebook(handle=part_rates)

In [5]:
# function to update average benefit for all
def aba_update(year):
    mcare_aba.data_source.data['Medicare'] = avg_ben_all[year]['Medicare']
    mcaid_aba.data_source.data['Medicaid'] = avg_ben_all[year]['Medicaid']
    ss_aba.data_source.data['Social Security'] = avg_ben_all[year]['Social Security']
    snap_aba.data_source.data['SNAP'] = avg_ben_all[year]['SNAP']
    ssi_aba.data_source.data['SSI'] = avg_ben_all[year]['SSI']
    vb_aba.data_source.data['Veterans Benefits'] = avg_ben_all[year]['Veterans Benefits']
    avg_ben_all_fig.title.text = 'Average Benefit by AGI Percentile for {}'.format(year)
    push_notebook(handle=aba_fig)

In [6]:
# function to update average benefit for participants
def abp_update(year):
    mcare_abp.data_source.data['Medicare'] = avg_ben_participants[year]['Medicare']
    mcaid_abp.data_source.data['Medicaid'] = avg_ben_participants[year]['Medicaid']
    ss_abp.data_source.data['Social Security'] = avg_ben_participants[year]['Social Security']
    snap_abp.data_source.data['SNAP'] = avg_ben_participants[year]['SNAP']
    ssi_abp.data_source.data['SSI'] = avg_ben_participants[year]['SSI']
    vb_abp.data_source.data['Veterans Benefits'] = avg_ben_participants[year]['Veterans Benefits']
    avg_ben_part_fig.title.text = 'Average Benefit for Participants by AGI Percentile for {}'.format(year)
    push_notebook(handle=abp_fig)

In [7]:
# function to update the total benefit by bin plot
def bin_total_bar_update(year, ben):
    ben_total_bar.data_source.data['top'] = ben_totals_bin[year][ben]
    ben_total_fig.title.text = 'Total {} Benefits by AGI Percentile Bin {}'.format(ben, year)
    push_notebook(handle=bin_total_handle)

In [8]:
# function to update the average benefit by bin plot
def bin_avg_bar_update(year, ben):
    ben_avg_bar.data_source.data['top'] = ben_avg_bin[year][ben]
    ben_avg_fig.title.text = 'Average {} Benefits by AGI Percentile Bin for {}'.format(ben, year)
    push_notebook(handle=bin_avg_handle)

###### Dropdowns for plots

In [9]:
# total benefits by year dropdown
total_dropdown = Dropdown(options=ben_name_list, value='Medicare', description='Benefit')

In [10]:
# Participation rates by year dropdown
year_dropdown = Dropdown(options=years, value=2014, description='Year')

In [11]:
# average benefits for all dropdown
year_dropdown2 = Dropdown(options=years, value=2014, description='Year')

In [12]:
# average benefits for participants dropdown
year_dropdown3 = Dropdown(options=years, value=2014, description='Year')

In [13]:
# total benefits by AGI bin dropdown
year_dropdown4 = Dropdown(options=years, value=2014, description='Year')
bin_total_dropdown = Dropdown(options=ben_name_list, value='Medicare', description='Benefit')

In [14]:
# average benefits by AGI dropdown
year_dropdown5 = Dropdown(options=years, value=2014, description='Year')
bin_avg_dropdown = Dropdown(options=ben_name_list, value='Medicare', description='Benefit')

### Calculations for benefits plots

In [15]:
# calculator used to create the data for plotting
calc_plots = Calculator(records=Records.cps_constructor(), policy=Policy())

You loaded data for 2014.
Your data include the following unused variables that will be ignored:
  total_ben
Tax-Calculator startup automatically extrapolated your data to 2014.


In [16]:
# dictionaries that will hold plot data
benefit_total_years = {'Medicare': [], 'Medicaid': [],
                       'Veterans Benefits': [], 'SNAP': [],
                       'SSI': [], 'Social Security': [],
                       'year': [year for year in range(2014, 2027)]}  # sum of each benefit by year
participation_rates = {}  # participation rates for each benefit by year
avg_ben_all = {}  # average benefit for everyone by AGI percentile
avg_ben_participants = {}  # average benefit by AGI percentile for just participants
ben_totals_bin = {}  # total benefits by year by AGI bin
ben_avg_bin = {}  # average benefits by year by AGI bin
# loop through each year to get data needed for benefits plots
for year in range(2014, 2027):
    calc_plots.advance_to_year(year)
    calc_plots.calc_all()
    # DataFrame to hold all the needed data
    ben_df = pd.DataFrame({'s006': calc_plots.records.s006,
                           'Medicare': calc_plots.records.mcare_ben,
                           'Medicaid': calc_plots.records.mcaid_ben,
                           'Veterans Benefits': calc_plots.records.vet_ben,
                           'SNAP': calc_plots.records.snap_ben,
                           'SSI': calc_plots.records.ssi_ben,
                           'Social Security': calc_plots.records.ss_ben,
                           'AGI': calc_plots.records.c00100})
    # create dummies for participation
    ben_df['mcare_part'] = np.where(ben_df['Medicare'] > 0, 1, 0)
    ben_df['mcaid_part'] = np.where(ben_df['Medicaid'] > 0, 1, 0)
    ben_df['vb_part'] = np.where(ben_df['Veterans Benefits'] > 0, 1, 0)
    ben_df['snap_part'] = np.where(ben_df['SNAP'] > 0, 1, 0)
    ben_df['ssi_part'] = np.where(ben_df['SSI'] > 0, 1, 0)
    ben_df['ss_part'] = np.where(ben_df['Social Security'] > 0, 1, 0)
    
    # initialize keys in certain dictionaries
    participation_rates[year] = {}
    avg_ben_all[year] = {}
    avg_ben_participants[year] = {}
    ben_totals_bin[year] = {}
    ben_avg_bin[year] = {}

    # loop through benefit names to add to dictionaries
    for i in range(0, len(ben_name_list)):
        ben = ben_name_list[i]
        ben_part = ben_part_names[i]
        # add weighted sum for each benefit to their list
        benefit_total_years[ben].append(weighted_sum(ben_df, ben))
        # add mean participation rates by benefit
        participation_rates[year][ben] = percentile(ben_df, ben_part, 100, 'AGI')
        participation_rates[year]['index'] = [i for i in range(1, 101)]
        # add average benefit by percentile for all
        avg_ben_all[year][ben] = percentile(ben_df, ben, 100, 'AGI')
        # add average benefit by percentile for participants
        sub_df = copy.deepcopy(ben_df[ben_df[ben_part] == 1])
        avg_ben_participants[year][ben] = percentile(sub_df, ben, 100, 'AGI')
        ben_totals_bin[year][ben] = percentile(ben_df, ben, 10, 'AGI', result_type='sum')
        ben_avg_bin[year][ben] = percentile(ben_df, ben, 10, 'AGI')

In [17]:
# add info to dictionaries as needed

benefit_total_years['year'] = years
benefit_total_years['top'] = [i * 0 for i in years]

avg_ben_all[2014]['index'] = participation_index
avg_ben_participants[2014]['index'] = participation_index

ben_totals_bin[2014]['index'] = bin_axis
ben_avg_bin[2014]['index'] = bin_axis
ben_totals_bin[2014]['top'] = ben_totals_bin[2014]['Medicaid']
ben_avg_bin[2014]['top'] = ben_avg_bin[2014]['Medicaid']

In [18]:
# create starting Column Data Sources
ben_total_cds = ColumnDataSource(benefit_total_years)
participation_rates_cds = ColumnDataSource(participation_rates[2014])
avg_ben_all_cds = ColumnDataSource(avg_ben_all[2014])
avg_ben_participants_cds = ColumnDataSource(avg_ben_participants[2014])
ben_totals_bin_cds = ColumnDataSource(ben_totals_bin[2014])
ben_avg_bin_cds = ColumnDataSource(ben_avg_bin[2014])

# Total Benefits by Year

In [19]:
total_fig = figure(title='Total Benefit by Year',
                   y_range=(0.0, 1e12))
init_bar = total_fig.vbar(x='year', width=0.75, bottom=0, top='top', source=ben_total_cds)

hover = HoverTool(tooltips=[('Value', '@top{$ 0.00 a}')])
total_fig.add_tools(hover)

total_fig.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
total_fig.xaxis.minor_tick_line_color = None
total_fig.xaxis.axis_label = 'Year'
total_fig.yaxis.axis_label = 'Total Benefit'

In [41]:
value_handle = show(total_fig, notebook_handle=True)
interactive(total_bar_update, ben=total_dropdown)

# Participation Rates by Program

In [21]:
part_fig = figure(title='Participation Rates by AGI Percentile for 2014')
mcare_part = part_fig.line(x='index', y='Medicare', legend='Medicare ', line_width=2,
                           source=participation_rates_cds)
mcaid_part = part_fig.line(x='index', y='Medicaid', legend='Medicaid ', line_width=2, color='red',
                           source=participation_rates_cds)
ss_part = part_fig.line(x='index', y='Social Security', legend='Social Security ', line_width=2, color='green',
                        source=participation_rates_cds)
snap_part = part_fig.line(x='index', y='SNAP', legend='SNAP ', line_width=2, color='orange',
                          source=participation_rates_cds)
ssi_part = part_fig.line(x='index', y='SSI', legend='SSI ', line_width=2, color='purple',
                         source=participation_rates_cds)
vb_part = part_fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ', line_width=2, color='pink',
                        source=participation_rates_cds)
hover_part = HoverTool(tooltips=[('Rate', '$y{0.0%}'), ('Percentile', '@index%')])
part_fig.add_tools(hover_part)

part_fig.yaxis.axis_label = 'Rate'
part_fig.xaxis.axis_label = 'AGI Percentile'

In [40]:
part_rates = show(part_fig, notebook_handle=True)
interactive(participation_rate_update, year=year_dropdown)

# Average Benefits for All

In [23]:
avg_ben_all_fig = figure(title='Average Benefit by AGI Percentile for 2014')
mcare_aba = avg_ben_all_fig.line(x='index', y='Medicare', legend='Medicare ', line_width=2,
                                 source=avg_ben_all_cds)
mcaid_aba = avg_ben_all_fig.line(x='index', y='Medicaid', legend='Medicaid ', line_width=2, color='red',
                                 source=avg_ben_all_cds)
ss_aba = avg_ben_all_fig.line(x='index', y='Social Security', legend='Social Security ', line_width=2, color='green',
                              source=avg_ben_all_cds)
snap_aba = avg_ben_all_fig.line(x='index', y='SNAP', legend='SNAP ', line_width=2, color='orange',
                                source=avg_ben_all_cds)
ssi_aba = avg_ben_all_fig.line(x='index', y='SSI', legend='SSI ', line_width=2, color='purple',
                               source=avg_ben_all_cds)
vb_aba = avg_ben_all_fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ', line_width=2, color='pink',
                              source=avg_ben_all_cds)
hover_aba = HoverTool(tooltips=[('Benefit', '$y{$0,0.00}'), ('Percentile', '@index%')])
avg_ben_all_fig.add_tools(hover_aba)

avg_ben_all_fig.xaxis.axis_label = 'Percentile'
avg_ben_all_fig.yaxis.axis_label = 'Average Benefit'

In [39]:
aba_fig = show(avg_ben_all_fig, notebook_handle=True)
interactive(aba_update, year=year_dropdown2)

# Average Benefit for Participants

In [25]:
avg_ben_part_fig = figure(title='Average Benefit for Participants by AGI Percentile for 2014',
                          width=800, y_range=(0, 45000))
mcare_abp = avg_ben_part_fig.line(x='index', y='Medicare', legend='Medicare ',
                                  line_width=2,
                                  source=avg_ben_participants_cds)
mcaid_abp = avg_ben_part_fig.line(x='index', y='Medicaid', legend='Medicaid ',
                                  line_width=2, color='red',
                                  source=avg_ben_participants_cds)
ss_abp = avg_ben_part_fig.line(x='index', y='Social Security', legend='Social Security ',
                               line_width=2, color='green',
                               source=avg_ben_participants_cds)
snap_abp = avg_ben_part_fig.line(x='index', y='SNAP', legend='SNAP ',
                                 line_width=2, color='orange',
                                 source=avg_ben_participants_cds)
ssi_abp = avg_ben_part_fig.line(x='index', y='SSI', legend='SSI ',
                                line_width=2, color='purple',
                                source=avg_ben_participants_cds)
vb_abp = avg_ben_part_fig.line(x='index', y='Veterans Benefits', legend='Veterans Benefits ',
                               line_width=2, color='pink',
                               source=avg_ben_participants_cds)
hover_abp = HoverTool(tooltips=[('Benefit', '$y{$0,0.00}'), ('Percentile', '@index%')])
avg_ben_part_fig.add_tools(hover_abp)

avg_ben_part_fig.legend.orientation = 'horizontal'
avg_ben_part_fig.legend.location = 'top_center'

avg_ben_part_fig.xaxis.axis_label = 'Percentile'
avg_ben_part_fig.yaxis.axis_label = 'Average Benefit'

In [38]:
abp_fig = show(avg_ben_part_fig, notebook_handle=True)
interactive(abp_update, year=year_dropdown3)

# Benefit Total By AGI Bin by Year

In [27]:
ben_total_fig = figure(title='Total Medicaid Benefits by AGI Percentile Bin for 2014',
                       x_range=bin_axis)
ben_total_bar = ben_total_fig.vbar(x='index', width=0.75, bottom=0, top='top', source=ben_totals_bin_cds)

hover_btb = HoverTool(tooltips=[('Value', '@top{$ 0.00 a}')])
ben_total_fig.add_tools(hover_btb)

ben_total_fig.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
ben_total_fig.xaxis.minor_tick_line_color = None

ben_total_fig.xaxis.axis_label = 'AGI Percentile Bin'
ben_total_fig.yaxis.axis_label = 'Total Benefits'

In [36]:
bin_total_handle = show(ben_total_fig, notebook_handle=True)
interactive(bin_total_bar_update, year=year_dropdown4, ben=bin_total_dropdown)

# Average Benefit by AGI Bin and Year
Among Participants

In [29]:
ben_avg_fig = figure(title='Average Medicaid Benefits by AGI Percentile Bin for 2014',
                     x_range=bin_axis)
ben_avg_bar = ben_avg_fig.vbar(x='index', width=0.75, bottom=0, top='top', source=ben_avg_bin_cds)

hover_bab = HoverTool(tooltips=[('Value', '@top{$ 0.00 a}')])
ben_avg_fig.add_tools(hover_bab)

ben_avg_fig.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
ben_avg_fig.xaxis.minor_tick_line_color = None
ben_avg_fig.xaxis.axis_label = 'AGI Percentile Bin'
ben_avg_fig.yaxis.axis_label = 'Average Benefits'

In [37]:
bin_avg_handle = show(ben_avg_fig, notebook_handle=True)
interactive(bin_avg_bar_update, year=year_dropdown5, ben=bin_avg_dropdown)

# Sample Reforms

In [31]:
# baseline calculator
pol = Policy()
rec = Records.cps_constructor()
calc = Calculator(policy=pol, records=rec)
calc.advance_to_year(2018)
calc.calc_all()

You loaded data for 2014.
Your data include the following unused variables that will be ignored:
  total_ben
Tax-Calculator startup automatically extrapolated your data to 2014.


## Repeal SSI, SNAP, Other Benefits

In [32]:
# reform calculator
pol2 = Policy()
reform = {
    2018: {'_BEN_Benefit_switch': [[False, False, True, True, True, True, False]]}
}
pol2.implement_reform(reform)
rec2 = Records.cps_constructor()
calc2 = Calculator(policy=pol2, records=rec2)
calc2.advance_to_year(2018)
calc2.calc_all()

You loaded data for 2014.
Your data include the following unused variables that will be ignored:
  total_ben
Tax-Calculator startup automatically extrapolated your data to 2014.


In [33]:
baseline_benefits = (calc.records.ben_total * calc.records.s006).sum()
reform_benefits = (calc2.records.ben_total * calc2.records.s006).sum()
change = reform_benefits - baseline_benefits
print ('Baseline Benefits: {:18.2f}'.format(baseline_benefits))
print ('Reform Benefits: {:20.2f}'.format(reform_benefits))
print ('-' * 37)
print ('Change in Benefits: {:17.2f}'.format(change))

Baseline Benefits:   2163269869190.04
Reform Benefits:     2020391727641.31
-------------------------------------
Change in Benefits:  -142878141548.73


## Apply Welfare Multiples

In [34]:
# reform calculator
pol3 = Policy()
reform2 = {
    2018: {
        '_BEN_Vet_mult': [0.95],
        '_BEN_Medicaid_mult': [0.30],
        '_BEN_Medicare_mult': [0.75]
    }
}
pol3.implement_reform(reform2)
rec3 = Records.cps_constructor()
calc3 = Calculator(policy=pol3, records=rec3)
calc3.advance_to_year(2018)
calc3.calc_all()

You loaded data for 2014.
Your data include the following unused variables that will be ignored:
  total_ben
Tax-Calculator startup automatically extrapolated your data to 2014.


In [35]:
total_benefits = (calc3.records.ben_total * calc3.records.s006).sum()
mult_benefits = (calc3.records.ben_welfare * calc3.records.s006).sum()
change = mult_benefits - total_benefits
print ('Total Benefits: {:26.2f}'.format(total_benefits))
print ('Benefits After Multiples: {:.2f}'.format(mult_benefits))
print ('-' * 46)
print ('Difference: {:30.2f}'.format(change))

Total Benefits:           2163269869190.04
Benefits After Multiples: 1737091861659.44
----------------------------------------------
Difference:               -426178007530.60
